# Agent Streaming and Responses

Master model invocation methods, streaming modes, and structured outputs.

**What you'll learn:**
- invoke: Single message or conversation list
- stream: Real-time chunk-by-chunk output
- batch: Parallel processing of multiple inputs
- Agent streaming: messages, updates, values modes
- Structured output: Type-safe responses with Pydantic

## Invocation Comparison

| Method | Use Case | Returns |
|--------|----------|--------|
| **invoke** | Single request | Complete response |
| **stream** | Real-time display | Iterator of chunks |
| **batch** | Multiple requests | List of responses |

In [ ]:
import sys
sys.path.append('D:/Courses/Udemy/AI Agent Projects')

import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3
from scripts import base_tools

In [ ]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

## Model Invocation Methods

In [ ]:
# Single message
response = model.invoke("Why do parrots have colorful feathers?")
response

In [ ]:
# Dictionary format
conversation = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Translate: I love programming."},
    {"role": "assistant", "content": "Translation complete."},
    {"role": "user", "content": "Translate: I love AI."}
]

response = model.invoke(conversation)
response

In [ ]:
# Message objects
conversation = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("What is AI?"),
    AIMessage("AI is artificial intelligence."),
    HumanMessage("What is ML?")
]

response = model.invoke(conversation)
response

## Stream

In [ ]:
# Basic streaming
for chunk in model.stream("Why do parrots have colorful feathers?"):
    print(chunk.content, end="|", flush=True)

In [ ]:
# Construct full AIMessage from chunks
full = None
for chunk in model.stream("What color is the sky?"):
    full = chunk if full is None else full + chunk
    print(full.content)

print("\nFinal:", full.content)

## Batch

In [ ]:
# Batch multiple requests
responses = model.batch([
    "Why do parrots have colorful feathers?",
    "How do airplanes fly?",
    "What is quantum computing?"
])

for i, response in enumerate(responses, 1):
    print(f"{i}. {response.content[:50]}...")

In [ ]:
# Batch with max concurrency
responses = model.batch(
    [
        "Question 1",
        "Question 2",
        "Question 3",
        "Question 4",
        "Question 5"
    ],
    config={'max_concurrency': 2}
)

len(responses)

## Agent Invocation

In [ ]:
conn = sqlite3.connect("db/streaming_agent.db", check_same_thread=False)
checkpointer = SqliteSaver(conn)

agent = create_agent(
    model=model,
    tools=[base_tools.web_search],
    checkpointer=checkpointer
)

In [ ]:
# Standard invoke
response = agent.invoke(
    {'messages': [HumanMessage('What is AI?')]},
    config={'configurable': {'thread_id': 'invoke_session'}}
)

response['messages'][-1].content

## Agent Streaming Modes

In [ ]:
# Stream mode: messages
for chunk in agent.stream(
    {'messages': [HumanMessage('Search for tech news')]},
    stream_mode='messages',
    config={'configurable': {'thread_id': 'stream_msg'}}
):
    print(chunk)

In [ ]:
# Stream mode: updates
for chunk in agent.stream(
    {'messages': [HumanMessage('Search for AI news')]},
    stream_mode='updates',
    config={'configurable': {'thread_id': 'stream_upd'}}
):
    print(chunk)

In [ ]:
# Stream mode: values
for step, chunk in enumerate(agent.stream(
    {'messages': [HumanMessage('What is ML?')]},
    stream_mode='values',
    config={'configurable': {'thread_id': 'stream_val'}}
)):
    print(f"Step {step}: {len(chunk.get('messages', []))} messages")

## Structured Output

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional
from langchain.agents.structured_output import ToolStrategy

class FinancialAnalysis(BaseModel):
    company: str = Field(description="Company name")
    stock_symbol: str = Field(description="Stock ticker")
    current_price: Optional[str] = Field(description="Current price", default=None)
    analysis: str = Field(description="Brief analysis")
    recommendation: str = Field(description="Buy/Hold/Sell")

agent = create_agent(
    model=model,
    tools=[base_tools.web_search],
    response_format=ToolStrategy(FinancialAnalysis)
)

response = agent.invoke({
    'messages': [HumanMessage('Analyze Tesla stock')]
})

response['structured_response']

In [ ]:
# Access structured data
data = response['structured_response']
print(f"Company: {data.company}")
print(f"Symbol: {data.stock_symbol}")
print(f"Recommendation: {data.recommendation}")

In [ ]:
# Exercise: Test different invocation methods
